In [155]:
# Instalación de paquetes necesarios (descomentar de ser necesario)

# !pip install numpy -q
# !pip install pandas -q
# !pip install matplotlib -q
# !pip install seaborn -q
# !pip install scikit-learn -q
# !pip install joblib -q

# imports

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [156]:
# Cargar el dataset
df_m = pd.read_excel('Mortalidad_infantil.xlsx')

# Visualizar las primeras filas
print("Primeras filas del dataset:")
display(df_m.head())

Primeras filas del dataset:


,ANIO,LOCALIDAD,N_MUERTES,NACIDOS_VIVOS,TASA POR 1000
0,2020,Antonio Nariño,8,995,8.0
1,2020,Barrios Unidos,13,1276,10.2
2,2020,Bosa,65,8508,7.6
3,2020,Chapinero,12,1422,8.4
4,2020,Ciudad Bolívar,88,8731,10.1


In [157]:
print(df_m["LOCALIDAD"].unique())
print(len(df_m["LOCALIDAD"].unique()))

['Antonio Nariño' 'Barrios Unidos' 'Bosa' 'Chapinero' 'Ciudad Bolívar'
 'Distrito' 'Engativá' 'Fontibón' 'Kennedy' 'La Candelaria' 'Los Mártires'
 'Puente Aranda' 'Rafael Uribe Uribe' 'San Cristóbal' 'Santa Fe'
 'Sin Información' 'Suba' 'Sumapaz' 'Teusaquillo' 'Tunjuelito' 'Usaquén'
 'Usme']
22


In [158]:
df_m["ANIO"] = df_m["ANIO"].replace("2024p", 2024)
print(df_m["ANIO"].unique())


[2020 2021 2022 2023 2024]


C:\Users\diego\AppData\Local\Temp\ipykernel_27052\4180279848.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_m["ANIO"] = df_m["ANIO"].replace("2024p", 2024)


In [159]:
df_m.dropna(inplace=True)
# eliminar localidades no deseadas
to_remove = ["Sin Información", "Distrito","Sumapaz"]
mask = df_m["LOCALIDAD"].astype(str).str.strip().isin(to_remove)
print(f"Filas a eliminar: {mask.sum()}")
df_m = df_m[~mask].copy()

Filas a eliminar: 10


In [160]:
print(df_m["LOCALIDAD"].unique())
print(len(df_m["LOCALIDAD"].unique()))

['Antonio Nariño' 'Barrios Unidos' 'Bosa' 'Chapinero' 'Ciudad Bolívar'
 'Engativá' 'Fontibón' 'Kennedy' 'La Candelaria' 'Los Mártires'
 'Puente Aranda' 'Rafael Uribe Uribe' 'San Cristóbal' 'Santa Fe' 'Suba'
 'Teusaquillo' 'Tunjuelito' 'Usaquén' 'Usme']
19


In [161]:
# Cargar el dataset
df_pm = pd.read_csv("pm2-5bogota.csv")

# Visualizar las primeras filas
print("Primeras filas del dataset:")
display(df_pm.head())

Primeras filas del dataset:


,Periodo,MOV,CBV,MAM,JAZ,PTE,MOV2,SCR,CSE,USM,...,CDAR,BOL,TUN,FTB,USQ,GYR,BOS,COL,KEN,LFR
0,2013-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,...,16.0,NaN,17.0,NaN,8.0,NaN,NaN,NaN,30.0,13.0
1,2013-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,...,19.0,NaN,23.0,NaN,12.0,NaN,NaN,NaN,33.0,NaN
2,2013-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,...,15.0,NaN,17.0,NaN,7.0,NaN,NaN,NaN,30.0,12.0
3,2014-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,NaN,18.0,NaN,10.0,NaN,NaN,NaN,36.0,17.0
4,2014-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,...,29.0,NaN,28.0,NaN,17.0,NaN,NaN,NaN,41.0,28.0


In [162]:
# Filtrar entre 2020 y 2024

df_pm["Periodo"] = df_pm["Periodo"].astype(str)

df_pm["Año"] = df_pm["Periodo"].str[:4].astype(int)


df_pm = df_pm[(df_pm["Año"] >= 2020) & (df_pm["Año"] <= 2024)]

df_pm = df_pm.drop(columns=["Año"])

print(df_pm.head())
print(f"Filas restantes: {len(df_pm)}")

    Periodo  MOV  CBV   MAM  JAZ   PTE  MOV2   SCR   CSE  USM  ...  CDAR  BOL  \
75  2020-01  NaN  NaN  13.0  NaN  15.0   NaN  12.0  30.0  NaN  ...  13.0  NaN   
76  2020-02  NaN  NaN  17.0  NaN  27.0   NaN  17.0  39.0  NaN  ...  20.0  NaN   
77  2020-03  NaN  NaN  29.0  NaN  37.0   NaN  28.0  44.0  NaN  ...  33.0  NaN   
78  2020-04  NaN  NaN  13.0  NaN  19.0   NaN  13.0  20.0  NaN  ...  14.0  NaN   
79  2020-05  NaN  NaN   7.0  NaN  13.0   NaN   NaN  19.0  NaN  ...   NaN  NaN   

     TUN   FTB   USQ   GYR  BOS  COL   KEN   LFR  
75  15.0  20.0  16.0  15.0  NaN  NaN  24.0  17.0  
76  21.0  27.0  22.0  21.0  NaN  NaN  29.0  19.0  
77  32.0  37.0  33.0  30.0  NaN  NaN  39.0  34.0  
78  13.0  18.0  16.0  14.0  NaN  NaN  19.0   NaN  
79   9.0  12.0   8.0   8.0  NaN  NaN  13.0   NaN  

[5 rows x 21 columns]
Filas restantes: 60


In [163]:
promedios = {}
promedios["ANTONIO NARIÑO"] = ["TUN", "SCR", "JAZ"]
promedios["BARRIOS UNIDOS"] = ["CDAR"]
promedios["BOSA"] = ["BOS"]
promedios["CHAPINERO"] = ["MOV", "MAM"]
promedios["CIUDAD BOLÍVAR"] = ["CBV"]
promedios["ENGATIVÁ"] = ["BOL", "LFR"]
promedios["FONTIBÓN"] = ["FTB", "MOV2"]
promedios["KENNEDY"] = ["KEN", "CSE"]
promedios["LA CANDELARIA"] = ["MAM", "SCR"]
promedios["LOS MÁRTIRES"] = ["JAZ", "SCR", "MAM"]
promedios["PUENTE ARANDA"] = ["JAZ", "PTE"]
promedios["RAFAEL URIBE URIBE"] = ["TUN", "SCR", "USN"]
promedios["SAN CRISTÓBAL"] = ["SCR"]
promedios["SANTA FE"] = ["MAM", "SCR"]
promedios["SUBA"] = ["SUB", "COL", "GYR"]
promedios["TEUSAQUILLO"] = ["CDAR", "MOV", "MAM", "PTE"]
promedios["TUNJUELITO"] = ["TUN", "CSE"]
promedios["USAQUÉN"] = ["USQ"]
promedios["USME"] = ["USM"]



In [164]:
df_localidades = pd.DataFrame()
df_localidades["Periodo"] = df_pm["Periodo"]


df_pm_filled = df_pm.fillna(0) #  NaN por 0 

# Calcular promedios por localidad
for loc, estaciones in promedios.items():
    estaciones_existentes = [e for e in estaciones if e in df_pm_filled.columns]
    if len(estaciones_existentes) > 0:
        df_localidades[loc] = round(df_pm_filled[estaciones_existentes].mean(axis=1),1)
    else:
        df_localidades[loc] = 0

# Mostrar resultado
print(df_localidades.head())

    Periodo  ANTONIO NARIÑO  BARRIOS UNIDOS  BOSA  CHAPINERO  CIUDAD BOLÍVAR  \
75  2020-01             9.0            13.0   0.0        6.5             0.0   
76  2020-02            12.7            20.0   0.0        8.5             0.0   
77  2020-03            20.0            33.0   0.0       14.5             0.0   
78  2020-04             8.7            14.0   0.0        6.5             0.0   
79  2020-05             3.0             0.0   0.0        3.5             0.0   

    ENGATIVÁ  FONTIBÓN  KENNEDY  LA CANDELARIA  LOS MÁRTIRES  PUENTE ARANDA  \
75       8.5      10.0     27.0           12.5           8.3            7.5   
76       9.5      13.5     34.0           17.0          11.3           13.5   
77      17.0      18.5     41.5           28.5          19.0           18.5   
78       0.0       9.0     19.5           13.0           8.7            9.5   
79       0.0       6.0     16.0            3.5           2.3            6.5   

    RAFAEL URIBE URIBE  SAN CRISTÓBAL  SANTA

In [165]:
df_localidades.to_csv("pm2-5bogota_localidades.csv", index=False)

In [166]:
# diccionario de diccionarios con promedios anuales por localidad

df_localidades["Año"] = df_localidades["Periodo"].str[:4]

columnas_localidades = [col for col in df_localidades.columns if col not in ["Periodo", "Año"]]

promedios_anuales = round(df_localidades.groupby("Año")[columnas_localidades].mean(),1)

diccionario_promedios = promedios_anuales.to_dict(orient="index")

for año, localidades in diccionario_promedios.items():
    print(f"{año}: {localidades}\n")


2020: {'ANTONIO NARIÑO': 8.7, 'BARRIOS UNIDOS': 13.1, 'BOSA': 0.0, 'CHAPINERO': 6.3, 'CIUDAD BOLÍVAR': 0.0, 'ENGATIVÁ': 5.8, 'FONTIBÓN': 9.1, 'KENNEDY': 23.2, 'LA CANDELARIA': 12.0, 'LOS MÁRTIRES': 8.0, 'PUENTE ARANDA': 10.3, 'RAFAEL URIBE URIBE': 13.0, 'SAN CRISTÓBAL': 11.2, 'SANTA FE': 12.0, 'SUBA': 9.6, 'TEUSAQUILLO': 11.6, 'TUNJUELITO': 19.7, 'USAQUÉN': 13.3, 'USME': 0.0}

2021: {'ANTONIO NARIÑO': 11.9, 'BARRIOS UNIDOS': 12.9, 'BOSA': 0.0, 'CHAPINERO': 6.0, 'CIUDAD BOLÍVAR': 0.0, 'ENGATIVÁ': 11.9, 'FONTIBÓN': 8.5, 'KENNEDY': 23.3, 'LA CANDELARIA': 11.8, 'LOS MÁRTIRES': 11.4, 'PUENTE ARANDA': 15.1, 'RAFAEL URIBE URIBE': 12.6, 'SAN CRISTÓBAL': 11.6, 'SANTA FE': 11.8, 'SUBA': 8.7, 'TEUSAQUILLO': 11.2, 'TUNJUELITO': 19.6, 'USAQUÉN': 6.4, 'USME': 0.0}

2022: {'ANTONIO NARIÑO': 17.1, 'BARRIOS UNIDOS': 15.3, 'BOSA': 0.0, 'CHAPINERO': 14.5, 'CIUDAD BOLÍVAR': 19.8, 'ENGATIVÁ': 15.9, 'FONTIBÓN': 20.9, 'KENNEDY': 13.5, 'LA CANDELARIA': 14.9, 'LOS MÁRTIRES': 15.6, 'PUENTE ARANDA': 16.7, 'RAFAE

In [167]:
import pandas as pd

# Función auxiliar para obtener el promedio PM2.5
def obtener_pm25(fila):
    año = str(fila["ANIO"])
    localidad = fila["LOCALIDAD"].upper().strip()
    
    # Intentar buscar en el diccionario 
    if año in diccionario_promedios:
        for loc_dic in diccionario_promedios[año].keys():
            if loc_dic.upper().strip() == localidad:
                return diccionario_promedios[año][loc_dic]
    return None  

# Crear la nueva columna
df_m["pm2.5"] = df_m.apply(obtener_pm25, axis=1)

# Mostrar el resultado
print(df_m.head(10))


    ANIO       LOCALIDAD  N_MUERTES  NACIDOS_VIVOS  TASA POR 1000  pm2.5
0   2020  Antonio Nariño          8            995            8.0    8.7
1   2020  Barrios Unidos         13           1276           10.2   13.1
2   2020            Bosa         65           8508            7.6    0.0
3   2020       Chapinero         12           1422            8.4    6.3
4   2020  Ciudad Bolívar         88           8731           10.1    0.0
6   2020        Engativá         61           7036            8.7    5.8
7   2020        Fontibón         23           3117            7.4    9.1
8   2020         Kennedy         90          10916            8.2   23.2
9   2020   La Candelaria          2            236            8.5   12.0
10  2020    Los Mártires         10           1207            8.3    8.0


In [168]:
df_m.to_csv("mortalidadBogotaConPm2-5.csv", index=False)